In [ ]:
# precisa ter o tesnroflow versao 2.10 no máximo pq tiraram suporte de GPU no windows por algum motivo
#!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
#!pip install --upgrade pip
#!pip install "tensorflow<2.11"

In [1]:
import datetime as dt
#import wget
import os
import pandas as pd
import numpy as np
#from zipfile import ZipFile
import zipfile
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
#from keras.regularizers import l2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import SGD
import funcoes as f
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # or any {'0', '1', '2'}
#from tensorflow.keras.preprocessing import image_dataset_from_directory
#from sklearn.svm import SVC
#from sklearn.svm import LinearSVC
#import plotly.express as px
#from sklearn.ensemble import BaggingClassifier
#from sklearn.multiclass import OneVsRestClassifier
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
#import absl.logging
from tqdm.auto import tqdm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

#absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')
#tf.config.run_functions_eagerly(True)
#tf.keras.mixed_precision.set_global_policy("mixed_float16")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h")#, "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20
# numero de quantis pra usar no markov transition field. Precisa tunar.
quantis = 3
# pega número de timeframes por minuto ou segundo
timeframes_padronizado = tuple(f.timeframes_mesma_unidade(timeframes))
# tamanho das batches de treinamento
batch_size = 1024#512
# set seed pros resultados não variarem
seed = np.random.randint(99999)
seed = 777
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)
# pega tamanho da imagem (input)
img_shape = (40, 100, 3)#f.image_shape()
# porcentagem dos dados que vou usar pra treino e teste
pct_imagens_teste = 0.1
pct_imagens_validacao = 0.1
# numero de modelos pro ensemble
n_modelos=5
# proporção pra diminuir as imagens no momento da criação 1 é imagem inteira
img_size = 1.0
# investimento inicial em dolares
investimento_inicial = 1000
# fee em %
fee = 0.1

# Prepara os dados

### Baixa e corrige dados

In [ ]:
# baixa os dados e junta cada timeframe e junta em 1 csv por timeframe
# retorna None se arquivo processado já existe e timeframe se ele criou um arquivo do 0
lista_processados = []
for timeframe in timeframes:
    lista_processados.append(f.baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017))

In [ ]:
# insere linhas sem dados e preenche com 0
# só corrige os timeframes que foram processados anteriormente
lista_p_correcao = [item for item in lista_processados if item != None]
f.corrige_arquivos(lista_p_correcao)

### Cria janelas, GAF's e salva imagens
Imagens ficam localizadas na pasta dados e são separadas por clissificação de compra ou venda

In [3]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs_close = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1][["Close", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_close = tuple(dfs_close)

# faz a mesma coisa pros dados de volume
dfs_volume = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Volume"])[::-1][["Volume", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_volume = tuple(dfs_volume)

#dfs_close[0], dfs_volume[0]

In [4]:
#pasta = "teste"
#linhas = range(1,10)

# transforma janelas em numpy arrays salvas em um único arquivo .tfrecords
# isso é ruim pra dar shuffle depois
def gera_tfrecords_unico(linhas, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):

    path = f'./Dados/TFRecords/{pasta}/{pasta}.tfrecords'#.encode('utf-8')
    writer = tf.io.TFRecordWriter(path, options="ZLIB")
    for linha in tqdm(linhas):
        # cria numpy array com precos de fechamento e pega a timestamp e decisao (compra ou venda)
        df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)

        # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
        if decisao == None or (0 in df_janela):
            continue

        df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]

        # cria gaf images
        img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))

        # converte pra valor entre 0 e 1
        img_array = (img_array+1)/2
        
        
        # trnasforma decisao em numero
        if decisao == "short1":
            decisao = 1
        elif decisao == "long0":
            decisao = 0
        
        
        record_bytes = tf.train.Example(features=tf.train.Features(feature={
                    'timestamp' : tf.train.Feature(int64_list=tf.train.Int64List(value=[timestamp])),
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[decisao])),
                    "image": tf.train.Feature(float_list=tf.train.FloatList(value=img_array.flatten()))
            })).SerializeToString()
        writer.write(record_bytes)
    
    writer.close()
    img_shape = img_array.shape
    return img_shape

#gera_tfrecords(linhas, dfs_close, dfs_volume, lookback, quantis, pasta, img_size)

In [5]:
# separa treino, validacao e teste
ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])

# linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
# linhas de validação vem logo após
# linhas de treino são as restantes
linhas_teste = range(1, int(ultima_linha*pct_imagens_teste))
linhas_validacao = range(int(ultima_linha*pct_imagens_teste), int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)))
linhas_treino = range(int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)), ultima_linha+1)

print("qtd de linhas para teste", len(linhas_teste))
print("qtd de linhas para validacao", len(linhas_validacao))
print("qtd de linhas para treino", len(linhas_treino))

qtd de linhas para teste 282527
qtd de linhas para validacao 282528
qtd de linhas para treino 2260225


In [ ]:
# gera arquivos .tfrecords

# gera dados de teste
gera_tfrecords_unico(linhas_teste, dfs_close, dfs_volume, lookback, quantis, "teste", img_size)
# gera dados de validacao
gera_tfrecords_unico(linhas_validacao, dfs_close, dfs_volume, lookback, quantis, "validacao", img_size)
# gera dados de treino
linhas_treino = np.array(linhas_treino)
np.random.shuffle(linhas_treino)
gera_tfrecords_unico(linhas_treino, dfs_close, dfs_volume, lookback, quantis, "treino", img_size)

  0%|          | 0/282527 [00:00<?, ?it/s]

  0%|          | 0/282528 [00:00<?, ?it/s]

  0%|          | 0/2260225 [00:00<?, ?it/s]

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

...

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
C:\Users\Guibolirows\anaconda3\lib\site-packages\pyts\preprocessing\discretizer.py:168: 

## Cria dataset

In [3]:
def decode_fn(tfrecord, img_shape=img_shape):
    '''
    Decodifica arquivos .tfrecords para que seja acessado como um dicionário pelo tensorflow
    
    Args:
    tfrecord: Arquivo tfrecord lido com a função tf.data.TFRecordDataset
    img_shape: shape de numpy array
    '''
    return  tf.io.parse_single_example(
      # Data
      tfrecord,

      # Schema
      {
        'timestamp': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature(img_shape, tf.float32)})        

In [4]:
def pega_imagem(tfrecord):
    return decode_fn(tfrecord)["image"]

def pega_imagem_e_label(tfrecord):
    tfrecord = decode_fn(tfrecord)
    return tfrecord["image"], tfrecord["label"]

In [5]:
# fazer lista de arquivo e label
#pasta = "teste"

def pega_files_labels(pasta):
    # como pegar timestamps e labels depois
    data = tf.data.TFRecordDataset(f'./Dados/TFRecords/{pasta}/{pasta}.tfrecords',
                                   compression_type="ZLIB").map(decode_fn)
    lista = []
    for item in data:
        lista_temp = [item["timestamp"].numpy(), item["label"].numpy()]
        lista.append(lista_temp)
    return pd.DataFrame(lista, columns = ["timestamp", "label"])

#pega_files_labels(pasta)

In [6]:
def cria_batches(batch_size=batch_size, valid_data=False, test_data=False, train_data=False):
    # se for treino precisa embaralhar antes
    if train_data == True:
        print("Criando dados de treino")
        pasta = "treino"
        path = f"./Dados/TFRecords/{pasta}/{pasta}.tfrecords"
        # lê arquivos
        data = tf.data.TFRecordDataset(
            path, compression_type="ZLIB", num_parallel_reads=tf.data.AUTOTUNE).map(
            pega_imagem_e_label, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return data
    # se for validacao não embaralha
    elif valid_data == True:
        print("Criando dados de validacao")
        pasta = "validacao"
        path = f"./Dados/TFRecords/{pasta}/{pasta}.tfrecords"
        # le a array
        data = tf.data.TFRecordDataset(
            path, compression_type="ZLIB", num_parallel_reads=tf.data.AUTOTUNE).map(
            pega_imagem_e_label, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return data
    # Se for teste não tem labels
    elif test_data == True:
        print("Criando dados de teste")
        pasta = "teste"
        path = f"./Dados/TFRecords/{pasta}/{pasta}.tfrecords"
        # le a array
        data = tf.data.TFRecordDataset(
            path, compression_type="ZLIB", num_parallel_reads=tf.data.AUTOTUNE).map(
            pega_imagem, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return data
    else:
        print("Chame a função corretamente")
        return

## Modelo

In [7]:
# rede CNN
def CNN(img_shape=img_shape):
    tf.keras.backend.clear_session()
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(8, activation="linear"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    opt = Adagrad(learning_rate=0.1)
    model.compile(loss = "binary_crossentropy", optimizer=opt, metrics=["accuracy", 
                                                                        keras.metrics.Precision(), 
                                                                        keras.metrics.Recall(),
                                                                        keras.metrics.AUC()])
    return model

## Treino

In [8]:
def treina_modelo_CNN(num_modelo, img_shape, batch_size):
    model = CNN(img_shape)
    # precisa definir o checkpoint antes de começar cada CNN. Se não acaba usando um do outro
    checkpoint = ModelCheckpoint(f"./modelos/upgrade/modelo CNN {num_modelo}", monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='val_accuracy', patience=4, mode="max")
    tb = TensorBoard(f"./modelos/upgrade/logs")
    callbacks_list = [checkpoint, es, tb]
    # treina    
    model.fit(cria_batches(batch_size, train_data=True),  epochs=10, use_multiprocessing=True, callbacks=callbacks_list, validation_data=cria_batches(batch_size, valid_data=True)) #verbose=0
    model = keras.models.load_model(f"./modelos/upgrade/modelo CNN {num_modelo}")
    return model.evaluate(cria_batches(batch_size, valid_data=True))

In [ ]:
resultado_CNN = np.empty((n_modelos, 5))
for i in range(n_modelos):
    resultado_CNN[i] = treina_modelo_CNN(i, img_shape, batch_size)

Criando dados de treino
Criando dados de validacao
Epoch 1/10
    168/Unknown - 30s 168ms/step - loss: 0.7315 - accuracy: 0.5017 - precision: 0.4952 - recall: 0.2767 - auc: 0.4995

In [7]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
%reload_ext tensorboard
%tensorboard --logdir f"./modelos/upgrade/logs"